In [35]:

from gibbs.dataclass import GibbsResult,get_results
import numpy as np
from tqdm import trange,tqdm
from gibbs.utils import simple_purify_hamiltonian,printarray,spectral_dec, identity_purification, expected_state
import matplotlib.pyplot as plt
from gibbs.learning.bayesian_learning import BayesianLearning
from gibbs.learning.constraint_matrix import ConstraintMatrixFactory
from gibbs.learning.klocal_pauli_basis import KLocalPauliBasis
from gibbs.plots.plots import compare_preparations, axis_locality,preparation
import plotly.express as px
from qiskit.quantum_info import state_fidelity,partial_trace

%load_ext autoreload
%autoreload 2
# gresult = get_results("../scripts_lap/job_17_qasm_ising_nocontrol")[0]
# gresult = get_results("../saved_simulations/turbo/simplerandomcfields")[0]
# gresult = get_results("../saved_simulations/turbo/really_long_evolution")[0]
gresult = get_results("../scripts_lap/job_18_qasm_ising_nocontrol")[0]

# print(gresult.betas)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
timestep = -1
print(gresult.betas[timestep])
# preparation(gresult,timestep).show()
non_local = np.abs(gresult.extend_locality(3).cfaulties[timestep][gresult.local_size(2):])
# px.bar(non_local).show()
print(f"The local error ammounts to {np.linalg.norm(gresult.cfaulties[timestep] - gresult.coriginal*gresult.betas[timestep])} and the non local errors ammount to {np.linalg.norm(non_local)}.")
rhotilde = partial_trace(gresult.state_vector(timestep),range(gresult.num_qubits,2*gresult.num_qubits))
rho = expected_state(gresult.hamiltonian,gresult.betas[timestep])
eigvals,_ = np.linalg.eigh(KLocalPauliBasis(2,4).vector_to_pauli_op(gresult.cfaulties[timestep] - gresult.coriginal*gresult.betas[timestep]))
print(f"We get a fidelity of {gresult.fidelity(timestep)} and an operator distance of {np.linalg.norm(rho-rhotilde,ord=2)} compared to {max(eigvals)},{min(eigvals)}")

0.64
The local error ammounts to 0.11021731279795423 and the non local errors ammount to 0.007291905667802296.
We get a fidelity of 0.9968081872891255 and an operator distance of 0.018311854445610974 compared to 0.22388004687172672,-0.19810620172969645


In [3]:
idnty_pur = identity_purification(gresult.num_qubits)
df = {
    "Betas" : gresult.betas[1:],
    "Classic Infidelity" : 1- gresult.fidelity_evolution(True)[1:],
    "Identity Infidelity" : [1- state_fidelity(simple_purify_hamiltonian(b*gresult.hamiltonian),idnty_pur) for b in tqdm(gresult.betas)][1:],
}
px.scatter(df,x="Betas",y=["Classic Infidelity","Identity Infidelity"],title="Fidelity Evolution",log_y=True)

100%|██████████| 17/17 [00:00<00:00, 136.10it/s]
